# Data Availability (Replication) Microbenchmark

In [1]:
# Move one directory up, so we are in the repository root
import os

if not 'CHDIR_EXECUTED' in globals():  # Keep chdir idempotent, don't keep going up
    os.chdir('..')
    CHDIR_EXECUTED = True
    
!basename "$(pwd)"

ndn-compute


In [2]:
import time
from functools import wraps
from typing import Callable, TypeVar, Any

T = TypeVar('T')

def time_function(func: Callable[..., T]) -> Callable[..., tuple[T, float]]:
    @wraps(func)
    def wrapper(*args, **kwargs) -> tuple[T, float]:
        start_time = time.time()
        result = func(*args, **kwargs)
        end_time = time.time()
        
        print(f"{func.__name__} executed in {end_time - start_time:.6f} seconds")
        return result, (end_time - start_time)
    
    return wrapper

import contextlib
import sys

class DummyFile(object):
    def write(self, x): pass

# Credit: https://stackoverflow.com/a/2829036
@contextlib.contextmanager
def no_stdout():
    save_stdout = sys.stdout
    sys.stdout = DummyFile()
    yield
    sys.stdout = save_stdout

In [3]:
from ndn_compute_fs_creator import create_fs_from_directory
from ndn_compute_cluster_manager import run_ndn_compute_cluster, stop_ndn_compute_cluster
from ndn_compute_client import NdnComputeClient

In [4]:
@time_function
def do_compute(client):
    dataset = client.create_dataset("appA/events.log.jsonl")
    pred = lambda row: row['event_type'] == 'purchase' and row['device'] == 'tablet' and row['browser'] == 'safari'
    dataset.filter(pred).collect()

In [5]:
compute_times = list()

for num_copies in range(1, 8+1):
    !rm -rf generated_data/distributed
    !mkdir -p generated_data/distributed
    create_fs_from_directory(in_dir="generated_data/flat",
                             out_dir="generated_data/distributed",
                             num_partitions=16,
                             num_copies=num_copies,
                             chunk_size=32
                             )
    
    # not sure why docker desktop is freaking out about mounting :(
    print("Restarting Docker...")
    !docker desktop restart > /dev/null
    
    print("Creating containers...")
    with no_stdout():
        run_ndn_compute_cluster(num_workers=16, rebuild=False)
    
    client = NdnComputeClient('http://localhost:5214')
    result = do_compute(client)
    compute_times.append(result[1])

generated_data/flat/appA/events.log.jsonl
Restarting Docker...
Creating containers...
do_compute executed in 4.466535 seconds
generated_data/flat/appA/events.log.jsonl
Restarting Docker...
Creating containers...
do_compute executed in 2.636674 seconds
generated_data/flat/appA/events.log.jsonl
Restarting Docker...
Creating containers...
do_compute executed in 3.441089 seconds
generated_data/flat/appA/events.log.jsonl
Restarting Docker...
Creating containers...
do_compute executed in 3.854837 seconds
generated_data/flat/appA/events.log.jsonl
Restarting Docker...
Creating containers...
do_compute executed in 6.298819 seconds
generated_data/flat/appA/events.log.jsonl
Restarting Docker...
Creating containers...
do_compute executed in 3.956015 seconds
generated_data/flat/appA/events.log.jsonl
Restarting Docker...
Creating containers...
do_compute executed in 3.763629 seconds
generated_data/flat/appA/events.log.jsonl
Restarting Docker...
Creating containers...
do_compute executed in 6.343022 

In [6]:
compute_times

[4.4665350914001465,
 2.636674165725708,
 3.441088914871216,
 3.854836940765381,
 6.298819065093994,
 3.956015110015869,
 3.7636289596557617,
 6.343022108078003]

In [7]:
@time_function
def do_compute_2(client):
    dataset = client.create_dataset("appA/events.log.jsonl")
    big_op = dataset.transform(lambda df: df.apply(lambda row: sum([__import__("math").sin(__import__("random").random()) * __import__("math").cos(__import__("random").random()) for _ in range(333)]), axis=1))
    lens = big_op.map(lambda r: len(str(r)))
    lens.collect()

In [8]:
compute_times_2 = list()

for num_copies in range(1, 8+1):
    !rm -rf generated_data/distributed
    !mkdir -p generated_data/distributed
    create_fs_from_directory(in_dir="generated_data/flat",
                             out_dir="generated_data/distributed",
                             num_partitions=16,
                             num_copies=num_copies,
                             chunk_size=32
                             )
    
    # not sure why docker desktop is freaking out about mounting :(
    print("Restarting Docker...")
    !docker desktop restart > /dev/null
    
    print("Creating containers...")
    with no_stdout():
        run_ndn_compute_cluster(num_workers=16, rebuild=False)
    
    client = NdnComputeClient('http://localhost:5214')
    result = do_compute_2(client)
    compute_times_2.append(result[1])

generated_data/flat/appA/events.log.jsonl
Restarting Docker...
Creating containers...
do_compute_2 executed in 26.333024 seconds
generated_data/flat/appA/events.log.jsonl
Restarting Docker...
Creating containers...
do_compute_2 executed in 26.328101 seconds
generated_data/flat/appA/events.log.jsonl
Restarting Docker...
Creating containers...
do_compute_2 executed in 16.343083 seconds
generated_data/flat/appA/events.log.jsonl
Restarting Docker...
Creating containers...
do_compute_2 executed in 14.333760 seconds
generated_data/flat/appA/events.log.jsonl
Restarting Docker...
Creating containers...


Fault: <Fault 1: "<class 'Exception'>:fetch_segments failed, exhausted attempts to fetch the first segment">

In [ ]:
compute_times_2